# PySpark 101

### An introduction to distributed computing

October 2020

### Installing PySpark

Steps for Windows:
1. Make sure you have java with `java -version`
2. Make sure you have Python with `python --version`
3. Download Spark from http://spark.apache.org/downloads.html
4. Unzip files with tar xvzf spark-3.0.0-bin-hadoop2.7.tgz
5. Set environment varibles with (you need to be running as admin):
    - setx SPARK_HOME "%USERPROFILE%\Documents\spark\spark-3.0.0-bin-hadoop2.7" /M
    - setx HADOOP_HOME "%USERPROFILE%\Documents\spark\spark-3.0.0-bin-hadoop2.7" /M

### Distributed Computing
For processing vast volumes of data fast, we need to **scale out** instead of **scale up**.  

- **Cheaper**: Run large data on clusters of many nodes (i.e. smaller and cheaper machines.)

- **Faster**: It parallelizes and distributes computations.

- **Reliable**: If one node or process fails, its workload is assumed by other components in the system. (Also known as fault tolerance).

### Spark Apache
Open-source distributed cluster-computing framework. It extends the MapReduce model using RDDs (Resilient Distributed Datasets).

Available in Java, Scala, Python, R. (PySpark is the Python distribution.)

**Key Components:**

- **Distribution**: Distribute the data

- **Parallelism**: Perform subsets of the computation simultaneously

- **Fault tolerance**: Handle component failure

# Part 1: MapReduce

### MapReduce
Let’s remember that MapReduce is a programming model for processing big data sets with a parallel, distributed algorithm on a cluster. 

- **Map procedure**: Applies a function to each data point over a partition in parallel. 
Examples: `filter()`, `map()`
- **Reduce procedure**: Summary operation that returns one value from multiple values
Examples: `reduce()`, `sum()`, `count()`

<img src='img/example.png'>

In [ ]:
# It helps you find spark
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
sc = SparkContext() 
ss = SparkSession.builder.getOrCreate()

ss.catalog.clearCache()
sc.setLogLevel("OFF")

In [ ]:
sc

In [ ]:
# Read data as an RDD
rdd = sc.textFile("data/numbers.txt", 8) # Number of cores

In [ ]:
# See data on the computer
rdd.glom().collect()

Logical cores = (# of physical cores) x (# of threads that can run on each core)

`sysctl -n hw.ncpu`

`rdd.getNumPartitions()`

In [ ]:
# A note about lambda functions

def add_2(x):
    return x+2

add_2_lambda = lambda x: x+2

assert add_2(3) == add_2_lambda(3)

#### Now let's convert data to integers

In [ ]:
# Example of a map function
converted_rdd = rdd.map(lambda x: int(x))
converted_rdd.glom().collect()

<img src='img/string2int.png'>

#### Now let's filter to get numbers greater or equal to 8

In [ ]:
# Example of another map function
filtered_rdd = converted_rdd.filter(lambda x: x >= 8)
filtered_rdd.glom().collect()

<img src='img/greater8.png'>

#### Now let's add numbers

In [ ]:
# Example of a reduce function
filtered_rdd.reduce(lambda x, y: x+y)

**What's happening?** The rdd is adding two numbers in the same partition until there is only one number left. Then it adds two numbers in different partitions until there is only one numebr left (shuffling happens).

In [ ]:
# Example of another reduce function
rdd.count()

#### Part 1: MapReduce -- Summary
Sum of integers that are greater than or equal to 8

In [ ]:
# Read data as an RDD
rdd = sc.textFile("data/numbers.txt", 8) # Number of cores

converted_rdd = rdd.map(lambda x: int(x)) # Map function that converts to integers
filtered_rdd = converted_rdd.filter(lambda x: x >= 8) # Map function to filter numbers

filtered_rdd.glom().collect() # To see results in partitions

filtered_rdd.reduce(lambda x, y: x+y) # Reduce function to add integers

In [ ]:
# Other useful functions

rdd.getNumPartitions() # Get number of partitions
rdd.count() # Reduce function to count how many elements are in the original rdd
rdd.first() # Get first object

In [ ]:
rdd.saveAsTextFile("ex01_output") # Saves it into parquet files

# Part 2: RDD Operations

### RDDs (Resilient Distributed Datasets)

Abstraction of a distributed collection of items with operations and transformation applicable to the dataset.

They are:
- Distributed
- Immutable
- Resilient

### RDD Operations

- **Transformations**:
    - Perform functions against each element in an RDD and return a new RDD
    - *Lazy evaluation*: Operations are only evaluated when an action is requested
- **Actions**
    - Trigger a computation and return a value to the Spark driver

### RDD Operations - Transformations

<img src="img/transformations.png">

### RDD Operations - Actions

<img src="img/actions.png">

#### Excercise: Calculate the sum of the odd numbers

In [ ]:
sc.stop() # Don't forget to close your connection